# Wave to Spectrogram

* 2초 단위로 자른 wav file을 stft(Short-time Fourier Transfromation)를 이용하여 (time, frequency) domain으로 바꿈
* 바꾼 그림을 각각 그림 파일 (jpg)로 저장

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import imageio

import librosa
import librosa.display
import musdb

from IPython import display

In [ ]:
mus = musdb.DB(root_dir='./datasets/musdb18/')

In [ ]:
# load the training tracks
tracks = mus.load_mus_tracks(subsets=['train'])
print(type(tracks))
print(len(tracks))

In [ ]:
tracks[0].targets.keys()

In [ ]:
index = 0
print(tracks[index].name)

In [ ]:
# original track - mixture
display.Audio(tracks[index].audio.T, rate=44100)

In [ ]:
print(tracks[index].audio.T.shape)

In [ ]:
# if you want to listen in each stem source then uncomment them
# display.Audio(tracks[index].targets['vocals'].audio.T, rate=44100)
# display.Audio(tracks[index].targets['drums'].audio.T, rate=44100)
# display.Audio(tracks[index].targets['bass'].audio.T, rate=44100)
# display.Audio(tracks[index].targets['other'].audio.T, rate=44100)
# display.Audio(tracks[index].targets['accompaniment'].audio.T, rate=44100)

## Plot for short time

In [ ]:
# Separate to left and right channels
second = 10
left_wave = tracks[index].audio.T[0][:44100 * second]
left_wave /= max(abs(left_wave))
right_wave = tracks[index].audio.T[1][:44100 * second]
right_wave /= max(abs(right_wave))

In [ ]:
# Plot the each channel
plt.figure(figsize=[18, 3])
plt.plot(left_wave)

plt.figure(figsize=[18, 3])
plt.plot(right_wave)
plt.show()

In [ ]:
# Short-time Fourier Transform
left_stft = librosa.core.stft(left_wave, n_fft=2048, hop_length=512)
print(left_stft.shape)
print(type(left_stft[0, 0]))

In [ ]:
left_abs = abs(left_stft)
librosa.display.specshow(left_abs)
plt.colorbar()
plt.show()

librosa.display.specshow(librosa.amplitude_to_db(left_abs, ref=np.max))
plt.colorbar()
plt.show()

## Spectrogram for standard form

In [ ]:
min_level_db = -100
ref_level_db = 20

In [ ]:
def spectrogram(y):
  D = _stft(y)
  S = _amp_to_db(np.abs(D)) - ref_level_db
  return _normalize(S)

In [ ]:
def _stft(y):
  n_fft, hop_length, win_length = 2048, 512, 2048
  return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

In [ ]:
def _amp_to_db(x):
  return 20 * np.log10(np.maximum(1e-5, x))

In [ ]:
def _normalize(S):
  return np.clip((S - min_level_db) / -min_level_db, 0, 1)

In [ ]:
left_spec = spectrogram(left_wave)
print(left_spec.shape)

In [ ]:
plt.figure()
librosa.display.specshow(left_spec)
plt.colorbar()
plt.show()

In [ ]:
# data split
spectrogram_datadir = './datasets/spectrogram_jpg/train'
second = 2 # split in 2 seconds
for i, track in enumerate(tracks):
  number = int(track.audio.T.shape[1] / second / 44100)
  interval = second * 44100
  for j in range(number):
    print("# {} track: {}th part".format(i, j))
    mixtures = track.audio.T[:, j*interval:(j+1)*interval]
    vocals = track.targets['vocals'].audio.T[:, j*interval:(j+1)*interval]
    drums = track.targets['drums'].audio.T[:, j*interval:(j+1)*interval]
    basses = track.targets['bass'].audio.T[:, j*interval:(j+1)*interval]
    others = track.targets['other'].audio.T[:, j*interval:(j+1)*interval]
    accompaniments = track.targets['accompaniment'].audio.T[:, j*interval:(j+1)*interval]
    
    sources = [mixtures, vocals, drums, basses, others, accompaniments]
    left_concat = np.zeros(shape=[1025, 173])
    right_concat = np.zeros(shape=[1025, 173])
    for k, wave in enumerate(sources):
      left_wave = wave[0]
      right_wave = wave[1]
      
      left_spec = spectrogram(left_wave) # (1025, 173) shape
      right_spec = spectrogram(right_wave) # (1025, 173) shape
      
      if k == 0:
        left_concat = left_spec
        right_concat = right_spec
      else:
        left_concat = np.concatenate((left_concat, left_spec), axis=1)
        right_concat = np.concatenate((right_concat, right_spec), axis=1)
        
    display.clear_output(wait=True)      
    filename_l = 'track{}.part{}.left.jpg'.format(i, j)
    filename_r = 'track{}.part{}.right.jpg'.format(i, j)
    print(filename_l, filename_r)
    
    imageio.imwrite(os.path.join(spectrogram_datadir, filename_l), left_concat)
    imageio.imwrite(os.path.join(spectrogram_datadir, filename_r), right_concat)